# Entry for SwarmsHackathon 2024



## Install Swarms

In [ ]:
!pip3 install -U swarms

Import keys

In [ ]:
from google.colab import userdata
anthropic_api_key = userdata.get('ANTHROPIC_API_KEY')

## Devin like

This example requires the anthropic library which is not installed by default.

In [ ]:
!pip install anthropic

In [ ]:
from swarms import Agent, Anthropic, tool
import subprocess

# Model
llm = Anthropic(
    temperature=0.1,
    anthropic_api_key = anthropic_api_key
)

# Tools
@tool
def terminal(
    code: str,
):
    """
    Run code in the terminal.

    Args:
        code (str): The code to run in the terminal.

    Returns:
        str: The output of the code.
    """
    out = subprocess.run(
        code, shell=True, capture_output=True, text=True
    ).stdout
    return str(out)


@tool
def browser(query: str):
    """
    Search the query in the browser with the `browser` tool.

    Args:
        query (str): The query to search in the browser.

    Returns:
        str: The search results.
    """
    import webbrowser

    url = f"https://www.google.com/search?q={query}"
    webbrowser.open(url)
    return f"Searching for {query} in the browser."

@tool
def create_file(file_path: str, content: str):
    """
    Create a file using the file editor tool.

    Args:
        file_path (str): The path to the file.
        content (str): The content to write to the file.

    Returns:
        str: The result of the file creation operation.
    """
    with open(file_path, "w") as file:
        file.write(content)
    return f"File {file_path} created successfully."

@tool
def file_editor(file_path: str, mode: str, content: str):
    """
    Edit a file using the file editor tool.

    Args:
        file_path (str): The path to the file.
        mode (str): The mode to open the file in.
        content (str): The content to write to the file.

    Returns:
        str: The result of the file editing operation.
    """
    with open(file_path, mode) as file:
        file.write(content)
    return f"File {file_path} edited successfully."


# Agent
agent = Agent(
    agent_name="Devin",
    system_prompt=(
        "Autonomous agent that can interact with humans and other"
        " agents. Be Helpful and Kind. Use the tools provided to"
        " assist the user. Return all code in markdown format."
    ),
    llm=llm,
    max_loops="auto",
    autosave=True,
    dashboard=False,
    streaming_on=True,
    verbose=True,
    stopping_token="<DONE>",
    interactive=True,
    tools=[terminal, browser, file_editor, create_file],
    code_interpreter=True,
    # streaming=True,
)

# Run the agent
out = agent("Create a new file for a plan to take over the world.")
print(out)

Ran into a conversion error when using Anthropic with the OpenAI API format:
```
2024-06-01T18:05:32.355005+0000 Number of tools: 4
2024-06-01T18:05:32.357823+0000 Tools provided, Automatically converting to OpenAI function
2024-06-01T18:05:32.367085+0000 Tool: decorator
2024-06-01T18:05:32.368336+0000 Tool: decorator
2024-06-01T18:05:32.368846+0000 Tool: decorator
2024-06-01T18:05:32.369262+0000 Tool: decorator

---------------------------------------------------------------------------

TypeError                                 Traceback (most recent call last)

<ipython-input-12-74d5a5dcf2f4> in <cell line: 82>()
     80
     81 # Agent
---> 82 agent = Agent(
     83     agent_name="Devin",
     84     system_prompt=(

1 frames

/usr/local/lib/python3.10/dist-packages/swarms/tools/py_func_to_openai_func_str.py in get_openai_function_schema_from_func(function, name, description)
    433     if missing != set():
    434         missing_s = [f"'{k}'" for k in sorted(missing)]
--> 435         raise TypeError(
    436             f"All parameters of the function '{function.__name__}' without default values must be annotated. "
    437             + f"The annotations are missing for the following parameters: {', '.join(missing_s)}"

TypeError: All parameters of the function 'decorator' without default values must be annotated. The annotations are missing for the following parameters: 'func'

```